In [1]:
import sys
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision import transforms, models
from torchvision.transforms import v2

In [3]:
from pathlib import Path
# include ../../3_Datapreparation
project_dir = Path(os.getcwd()).resolve().parent
sys.path.append(str(project_dir / '1_HaGRID'))
from dataset import ClassificationDataset, DetectionDataset, HagridDataset